# **Exercícios**

Este *notebook* deve servir como um guia para a construção da sua própria análise exploratória de dados. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/).

---

# **Análise Exploratória de Dados de Logística**

## 1\. Contexto

Escreva uma breve descrição do problema.

## 2\. Pacotes e bibliotecas

In [1]:
!pip3 install geopandas;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.7 MB/s eta 0:00:00


In [2]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

## 3\. Exploração de dados

In [3]:
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json 

In [4]:
with open('deliveries.json', mode='r', encoding='utf8') as file:
   data = json.load(file)

In [5]:
df = pd.DataFrame(data)

In [6]:
df.head(n = 1)

,name,region,origin,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."


In [7]:
hub_origin_df = pd.json_normalize(df["origin"])
hub_origin_df.head(n = 1)

,lng,lat
0,-48.054989,-15.838145


In [8]:
df = pd.merge(left=df, right=hub_origin_df, how='inner', left_index=True, right_index=True)
df.head(n = 1)

,name,region,origin,vehicle_capacity,deliveries,lng,lat
0,cvrp-2-df-33,df-2,"{'lng': -48.05498915846707, 'lat': -15.8381445...",180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p...",-48.054989,-15.838145


In [9]:
df = df.drop("origin", axis = 1)
df = df[["name", "region", "lng", "lat", "vehicle_capacity", "deliveries"]]
df.head(n = 1)

,name,region,lng,lat,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."


In [10]:
df.rename(columns={"lng": "hub_lng", "lat": "hub_lat"}, inplace=True)
df.head(n = 1)

,name,region,hub_lng,hub_lat,vehicle_capacity,deliveries
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,"[{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'p..."


In [11]:
deliveries_exploded_df = df[["deliveries"]].explode("deliveries")
deliveries_exploded_df.head(n = 1)

,deliveries
0,"{'id': '313483a19d2f8d65cd5024c8d215cfbd', 'po..."


In [12]:
deliveries_normalized_df = pd.concat([
 pd.DataFrame(deliveries_exploded_df["deliveries"].apply(
 lambda record: record["size"])
).rename(columns={"deliveries": "delivery_size"}),
 pd.DataFrame(deliveries_exploded_df["deliveries"].apply(
 lambda record: record["point"]["lng"])
).rename(columns={"deliveries": "delivery_lng"}),
 pd.DataFrame(deliveries_exploded_df["deliveries"].apply(
 lambda record: record["point"]["lat"])
).rename(columns={"deliveries": "delivery_lat"}),
], axis= 1)
deliveries_normalized_df.head(n = 1)

,delivery_size,delivery_lng,delivery_lat
0,9,-48.116189,-15.848929


In [13]:
df = df.drop("deliveries", axis=1)
df = pd.merge(df, right=deliveries_normalized_df, how='right', left_index=True, right_index=True)
df.reset_index(inplace=True, drop=True)
df.head(n = 1)

,name,region,hub_lng,hub_lat,vehicle_capacity,delivery_size,delivery_lng,delivery_lat
0,cvrp-2-df-33,df-2,-48.054989,-15.838145,180,9,-48.116189,-15.848929


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636149 entries, 0 to 636148
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              636149 non-null  object 
 1   region            636149 non-null  object 
 2   hub_lng           636149 non-null  float64
 3   hub_lat           636149 non-null  float64
 4   vehicle_capacity  636149 non-null  int64  
 5   delivery_size     636149 non-null  int64  
 6   delivery_lng      636149 non-null  float64
 7   delivery_lat      636149 non-null  float64
dtypes: float64(4), int64(2), object(2)
memory usage: 38.8+ MB


In [15]:
df.region.unique()

array(['df-2', 'df-1', 'df-0'], dtype=object)

In [16]:
df.vehicle_capacity.unique()

array([180])

In [17]:
df.delivery_size.unique()

array([ 9,  2,  1,  7, 10,  8,  3,  5,  6,  4])

In [18]:
df.isna().any()

name                False
region              False
hub_lng             False
hub_lat             False
vehicle_capacity    False
delivery_size       False
delivery_lng        False
delivery_lat        False
dtype: bool